<a href="https://colab.research.google.com/github/Welwi/RL_typo/blob/master/RL_typo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, I am training a RL algorithm that will master two varying envs with varying complexity.

ENV 1: Cartpole
The goal is to balance a pole, portruding from a cart, in an upright position by only movign the base left or right. This is an env with a low-dimensional observation space.

ENV2: Pong
The goal is to beat the competition. The env has a high-dimensional observation space - learning directly from raw pixels.

In [2]:

!apt-get install -y xvfb python-opengl x11-utils > /dev/null 2>&1
!pip install gym pyvirtualdisplay scikit-video > /dev/null 2>&1

!pip install mitdeeplearning

     |████████████████████████████████| 2.1MB 4.6MB/s 
  Created wheel for mitdeeplearning: filename=mitdeeplearning-0.1.2-cp36-none-any.whl size=2114586 sha256=48db7795b5cf778f217b4f026667ae7f9a343053121a85a2c242658669a21a0a
  Stored in directory: /root/.cache/pip/wheels/27/e1/73/5f01c787621d8a3c857f59876c79e304b9b64db9ff5bd61b74
Successfully built mitdeeplearning


## Steps of RL probs in general:
1. Initialize the env and the agent: describe the different observations and actions the agent can make in the env.

2. Define the agent's memory: this will enable the agent to remember its past actions, observations and rewards

3. Define a reward function: describes the reward associated with an action or sequence of actions

4. Define the learning algorithm: this is used to reinforce the agent's good behavior and discourage the bad behaviors.

In [3]:
import tensorflow as tf
import numpy as np

import base64, io, time, gym
import IPython, functools

import matplotlib.pyplot as plt

from tqdm import tqdm

import mitdeeplearning as mdl

## PART 1: CARTPOLE

Gym is a toolkit that has several pre-defined environments for training and testing RL learning agents.

In Cartpole, the pole starts upright and the goal is to prevent it from falling. A reward of +1 is given for every timestep that the pole remains upright.
A reward of -1 is given if the the pole is more than 15 degrees from the vertical or if the cart moves more than 2.4 unites from the center of the track.

In [4]:
# Instantiating the cartpole env

env = gym.make('CartPole-v0')
env.seed(1)

[1]

Observations that help define the env:
1. cart position
2. cart velocity
3. pole angle
4. pole rotation rate

Actions that the agent can take:
- The agent can move either right or left.

This shows that this is a low-dimensional observation and action spaces.

In [14]:
# Checking the size of the space
n_observations = env.observation_space
print('Env has observation space =', n_observations)

# Checking the num of actions that the agent can take
n_actions = env.action_space.n
print('Num of possible actions that the agent can choose from =', n_actions)

Env has observation space = Box(4,)
Num of possible actions that the agent can choose from = 2


In [15]:
n_actions

2

### Defining the agent

In RL, a deep neural network defines the agent.
This network takes in an observation of the environment, and outputs the probability of taking each of the possible actions.
Since this is a low dimensional observation space, we can use a simple feed forward NN

In [16]:
# Defining the carpole agent
def create_carpole_model():

  model = tf.keras.models.Sequential([
                                      tf.keras.layers.Dense(units=32, activation='relu'),
                                      tf.keras.layers.Dense(units=n_actions, activation= None)
  ])

  return model

cartpole_model = create_carpole_model()

Defining a feed forward pass through the network (action function)

- takes observations as inputs
- does a forward pass through the model
- outputs the agent action


In [42]:
def choose_action(model, observation):

  # adding the batch dimension to the observation
  observation = np.expand_dims(observation, axis=0)

  # passing the observation through the model
  logits = model.predict(observation)

  # pass the probabilities through softmax to get true probability
  prob_weights = tf.nn.softmax(logits).numpy()

  # random selection of an action of an action from observation
  action = np.random.choice(n_actions, size=1, p=prob_weights.flatten())[0]

  return action

## Breaking it down

In [30]:
# resets the state of the env and returns an initial observation
observation = env.reset()

In [31]:
observation

array([ 0.02725216, -0.04481721, -0.04304738,  0.00151751])

In [32]:
observation.shape

(4,)

In [27]:
# adding batch dimension to the obs
observation = np.expand_dims(observation, axis=0)

In [29]:
observation.shape

(1, 1, 4)

In [23]:
# passing obs through the model
logits = cartpole_model(observation)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [24]:
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.00299553, -0.00624159]], dtype=float32)>

In [25]:
  # pass the probabilities through softmax to get true probability
  prob_weights = tf.nn.softmax(logits).numpy()

In [26]:
prob_weights

array([[0.50230926, 0.49769074]], dtype=float32)

In [35]:
prob_weights.flatten()

array([0.50230926, 0.49769074], dtype=float32)

In [37]:
prob_weights.shape

(1, 2)

In [38]:
prob_weights.flatten().shape

(2,)

In [39]:
action = np.random.choice(n_actions, size=1, p=prob_weights.flatten())

In [40]:
action

array([0])

In [41]:
action[0]

0

In [ ]:
action = np.random.choice(n_actions, size=1, p=prob_weights.flatten())[0]